# Basic homomorphic encryption
Let's start with some basic homomorphic encryption using exponentiation of a generator and modulo

The first step is to get a polynomial evaluated in the unencrypted domain, encrypt the result.
The second step is to evaluate it in the encrypted domain.
The third step is to verify whether they match, or not.

$$ p(x)= c0 + c1*x + c2*x^2 +c3*x^3 $$

The other way to represent this polynomial is by using co-factors, which uses the roots (solution) of the polynomial.

$$ p(x) = (x - r1) * (x - r2) * (x - r3) = 0 $$


In [63]:
from IPython.display import Markdown as md
import numpy as np
# Values for the user to input, if he wants to
generator = 7
field_prime = 241
field_prime = 7919

# x^3 - 18 x^2 + 80 x - 96 = 0
# Co-factors (roots) for this polynomial are [2,4,12]

# let's define a utility function to calculate coefficients when we know the roots
def calculate_coeff_from_roots(polynomial_roots : list[int]):
    calculated_coefficients = np.polynomial.polynomial.polyfromroots(polynomial_roots)
    return list(map(int,calculated_coefficients.astype(int))) # map back to a list of int

# Let's compute the coefficents from known roots and see what we get. 
# We'll use numpy's integrated function to do this
polynomial_roots = [10,3,5]
p_polynomial_coefficients = calculate_coeff_from_roots(polynomial_roots)
target_polynomial_coefficients = []

# change_variables = input("Do you want to change variables?")
# if change_variables == "yes":
#     generator = input("generator")

equation = "$$ p(x)= "

for index, coeff in enumerate(p_polynomial_coefficients):

    if index == 0:
        equation+= str(coeff)
    else:
        if coeff >= 0:
            equation+= (" + ")
        else:
            equation+= (" - ")
        equation+= str(abs(coeff))
        equation+= (" * x^{}".format(index))
               
equation+= (" $$")
md(equation)
#md("$$ p(x)= {} + {}*x + c2*x^2 $$".format(p_polynomial_coefficients[0],p_polynomial_coefficients[1]))

$$ p(x)= -150 + 95 * x^1 - 18 * x^2 + 1 * x^3 $$

In [64]:
# define the encryption function
# all operations defined on Galois field G(p), with p being a prime number
# encryption is done by exponentiation of a generator (a prime number too) to the power of the unencrypted value
# Now fortunately, the pow() function in Python has an embedded functionality to apply a modulo
#  and it even works on large numbers, by using reduction
def encrypt(unencrypted, gen=generator,modulo=field_prime):
    encrypted = pow(gen,unencrypted,modulo)
    #print("exponentiated to : ", encrypted)
    return encrypted
    
print("encrypt 5, with generator 7 and p 19: ",encrypt(5,gen=7,modulo=19))
print("encrypt 6, with generator 7 and p 19: ",encrypt(6,gen=7,modulo=19))
print("encrypt 334499, with generator 7 and p 19: ",encrypt(334499,gen=7,modulo=19))
print('let\'s what we get with negative number -5:',encrypt(-5,gen=7,modulo=19))
print('The negative number is the same thing as applying the exponentiation to the inverse multiplicative',encrypt(14,gen=7,modulo=19) )


encrypt 5, with generator 7 and p 19:  11
encrypt 6, with generator 7 and p 19:  1
encrypt 334499, with generator 7 and p 19:  11
let's what we get with negative number -5: 7
The negative number is the same thing as applying the exponentiation to the inverse multiplicative 11


## Now we see how we can do our first encryption of polynomial
The way to do it is first to compute the polynomial in the unencrypted domain for a given r(a scalar that is used to compute the value of the polynomial, e.g. p(r)), then encrypt this result using the method described above.
Then the prover computes it in the encrypted domain, returns it to the verifier.
Finally, the Verifier verifies the 2 values match.

One of the things that has been a little complicated to integrate is how to prove equality in the encrypted domain.
So, what do we mean by encrypted domain to start with?
We mean that arithmetic operations have a different meaning than in the unencryted domain (i.e. the normal way 
we perform arithmetic)

The encrypted domain has elements that are in the form $E(x)=(g^x\mod p)$, with g and p being chosen primes.

When 2 encrypted elements are added together, we in fact use a multiplier to compute that addition. This is because if we have $E(a)=(g^a (\mod p)), E(b)=(g^b (\mod p))$ as elements and we add them in the unencrypted domain we get $E(a+b) \equiv (g^{a+b} (\mod p)) \equiv (g^a * g^b (\mod p)) \equiv (g^a \mod p) * (g^b \mod p) \equiv (E(a) * E(b))$

For instance, in the unencrypted domain $1 + 5 = 6$, if we want to compute it in the encrypted domain with $g=3, p=7$. 
We start by computing 6 in the encrypted domain: $E(6) = 3^6 (\mod 7) = 1$
Then we compute $E(1) * E(5) = (3^5 \mod 7 * (3^1 \mod 7) = 5 * 3 (\mod 7) = 1$

Now when we multiply 2 encrypted elements what we really are doing is exponentiating them. This is because again we are dealing with exponentiated elements. So let's see what we get when we perform a multiplication in the encrypted domain. $E(a * b) = (g^{a*b} (\mod p)) = ((g^a)^b (\mod p))$. Rightly, you may be wondering if the multiplication in the encrypted domain is commutative, if in fact $((g^a)^b (\mod p)) = ((g^b)^a (\mod p))$ ?

Let's take an example, with a=4 and b=5, and again g=3, p=7. In the unencrypted domain $5 * 4 = 20$, if we then encrypt this value we get $3^{20} \mod 7 = 2$.

Now to the encrypted domain:

The first equation to check is $((g^a)^b (\mod p))$. So this is $(3^4)^5 (\mod 7) \equiv (3^4 \mod 7)^ 5 \mod 7) \equiv 4^5 \mod 7 = 2$ ! success
The second equation to check is $((g^b)^a (\mod p))$. So this is $(3^5)^4 (\mod 7) \equiv (3^5 \mod 7)^ 4 \mod 7) \equiv 5^4 \mod 7 = 2$ ! success again

The 2 things that could have made this not true is the fact we use modulo at each step combined with _the power of a power rule_.

A negative number in the encrypted domain is found by calculating the multiplicative modular inverse of that number in the modulo field and is represented by $g^{-1} \mod p$, which means $g * g^{-1} = 1 (\mod p)$. And as we have seen above with the power rule of a power rule, we can rewrite $g^{-2} \mod p$ as $(g^{-1})^2 (\mod p)$

For instance if we want to encrypt -2, we get 4 by virtue of $3^{-2} \mod 7 \equiv (3^{-1})^2) \mod7$. The modular multiplication inverse of 3 is 5 (because $3 * 5 mod 7 = 1$). So we can rewrite that last equation with $5^2 (\mod 7) = 4$. Case solved!


When we perform a division in the encryted domain, we have to use modulo division concept which is also based on the modular inverse but I will not explain it, because we it is not needed.

Finally, it is NOT possible have the equivalent of exponentiation of the unencrypted domain in the encrypted one.

In [65]:
# We have defined the polynomial earlier with its coefficients in p_polynomial_coefficients
# let's compute the polynomial p(r) value for a given r (not a root)
r=20
evaluated_poly = np.polynomial.polynomial.polyval(r, p_polynomial_coefficients)
print('the polynomial evaluated at r: ',evaluated_poly)
# Now let's encrypt this value and see what we get
encrypted_result = encrypt(int(evaluated_poly))
print('encrypted result: ',encrypted_result)

# The second part is to verify that we can get the same result when doing the computation in the encryted domain
# How we do this is by having the verifier (he chooses a secret value where to evaluate the polynomial) give the encrypted value
# of each x^i for the value r. In our case the prover needs the encrypted values for 11^3, 11^2, 11^1 and 11^0
# For now we just want to see it in action, we are not yet implementing the protocol for Zero Knowledge proof
encrypted_powers = []
for index, order in enumerate(p_polynomial_coefficients):
    encrypted_powers.append(encrypt(pow(r,index)))

# Now let's compute E (p(11)) = encrypted_powers[0]^coeff[0] * encrypted_powers[1]^coeff[1] * encrypted_powers[2]^coeff[2] * encrypted_powers[3]^coeff[3] 
# We apply modulo a few times along the way to reduce the chance of overflow
result = 1
for index,coeff in enumerate(p_polynomial_coefficients):
    result = (result * pow(encrypted_powers[index],int(coeff),field_prime)) % field_prime
print('the result is: ....',result)


the polynomial evaluated at r:  2550.0
encrypted result:  3788
the result is: .... 3788


In [66]:

# I think from here on out it would be good to have a verifier and a prover classes and clearly delineate the tasks they each
# must perform. As the functionality will evolve as we make the protocol more complex, the classes will be named 
# using the chapter number defining the protocol

# We will need random values from here on out, let's use the secrets module for it
import secrets

# There are some limitations with using the numpy package to evaluate the polynomial when using large numbers
# Because of this we will use a [package](https://mhostetter.github.io/galois/latest/tutorials/intro-to-prime-fields/#prime-field) that can do polynomial evaluation in a Galois Field
#import galois

# A few things are different from the example given in the preceding section
# The verifier knows a polynomial t(x), called the target polynomial
# The prover claims he knows a polynomial of order n p(x) that is divisible by t(x) (thus has t(x)'s roots as co-factors)
# The prover wants to keep p(x) secret, but wants to prove he knows it
# Thus the prover computes p(x)/t(x) = h(x), and will provide the value of p(s) and h(s) to the verifier
# The prover uses s that is provided by the verifier, and the computation occurs in the encrypted domain
# As is mentioned in the pdf document, we cannot exponentiate in the encrypted domain
# That is why the verifier must provide the encrypted values of s for all orders of the polynomial
# That is E(s^0), E(s^1), ... , E(s^n), where n is the polynomial order
# Finally once the prover returns p(s) and h(s) to the verifier, the latter computes p(s)/t(s) and if it 
# matches p(s), then he knows it's correct (very bad protocol for now, but it'll get more secure)

# Because we are going to implement multiple version of the protocol, I'll use the protocol module
# There's a good explanation how it works at: https://idego-group.com/blog/2023/02/21/we-need-to-talk-about-protocols-in-python/
from typing import Protocol

from abc import ABC, abstractmethod

class Verifier(ABC):
    def __init__(self, target_polynomial_coefficients: list[int], max_order_p_poly: int, field_prime:int = field_prime, generator:int = generator):
        super().__init__()
        self.encrypted_powers = []
        self.field_prime = field_prime
        self.generator = generator
        self.target_polynomial_coefficients = target_polynomial_coefficients
        self.target_desc_coeffs = list(reversed(target_polynomial_coefficients))
        self.s : int
        self.max_order_p_poly = max_order_p_poly
    
    @abstractmethod
    def calculate_encrypted_powers(self):
        ...
    @abstractmethod
    def evaluate_unencrypted_target_polynomial(self):
        ...

    

class Verifier_334(Verifier):
# The verifier's tasks are as follows:
# (a) sample a random value s, i.e., secret
# (b) calculate encryptions of s for all powers i in 0, 1, ..., d, i.e.: E(si) = g^s^i
# (c) evaluate unencrypted target polynomial with s: t(s)
# (d) encrypted powers of s are provided to the prover: E(s0), E(s1), ..., E(sd)
# (e) The last step for the verifier is to checks that p = t(s) · h in encrypted space: 
# g^p =(g^h)^t(s) ⇒ g^p = g^t(s)·h

    def __init__(self, target_polynomial_coefficients: list[int], max_order_p_poly:int, field_prime:int = field_prime, generator:int = generator):
        #super().__init__(target_polynomial_coefficients, max_order_p_poly, field_prime, generator)
        super().__init__(target_polynomial_coefficients=target_polynomial_coefficients,max_order_p_poly=max_order_p_poly,field_prime=field_prime,generator=generator)
        self.s = secrets.randbelow(self.field_prime) # (a)
        #print(f'secret = {self.s}')
    
    def calculate_encrypted_powers(self):
        # (b) & (d)
        for index in range(self.max_order_p_poly):
            self.encrypted_powers.append(encrypt(pow(self.s,index)))

    def evaluate_unencrypted_target_polynomial(self):
        self.evaluated_poly = int(np.polynomial.polynomial.polyval(self.s, self.target_polynomial_coefficients))
        print(f'the polynomial evaluated at s = {self.s} is {evaluated_poly} ')

    def set_secret(self, s:int):
        self.s = s

    def check_encrypted_t_h_p(self, encrypted_h:int, encrypted_p:int) -> bool:
        # (e) 
        return (pow(encrypted_h,self.evaluated_poly,self.field_prime) == encrypted_p)


# let's exercise this new class
# target poly is (x-1)(x-2), which is equivalent to 2 -3x +x^2
# myVerifier =  Verifier_334([2,-3,1],max_order_p_poly=4)
# print(myVerifier.target_polynomial_coefficients)
# myVerifier.evaluate_unencrypted_target_polynomial()
# myVerifier.calculate_encrypted_powers()

# # just to make sure everything works fine, let's test this implementation with the same values we used in the 
# # preceding block
# myValidatedVerifier = Verifier_334(p_polynomial_coefficients, max_order_p_poly=4)
# myValidatedVerifier.set_secret(20)
# print(myValidatedVerifier.target_polynomial_coefficients)
# myValidatedVerifier.evaluate_unencrypted_target_polynomial()

# it is not working, the reason is that any value used in the computation of the GF must be in the finite field.
# so what we see is that the evaluation of the polynomial in the first implementation returns 2550 for r=20
# With the GF, the evaluated value is 140, which is 2550 mod 241! For this test we have been using 241 as the field prime
# How can we explain that the prover's result matches the verifier's then? 
# it seems to me that there is a good chance that is because the power of r values are outside of the field limit
# So, if we take a field prime that is big enough, the results will match. Let's see if this works: new field prime = 7919
# Result is that it works indeed! But I think the field prime modulo should be used on the powers of r too.
# This seems to give different results when reverting back to 241 as field prime... mystery to be solved 
# when implementing the prover. 
# Ok, so after thinking about this more, here is the resolution for this problem. Whenever computations occur in the
# non-encrypted domain, one must NOT apply the field restiction (modulo) until we go to the encrypted domain.
# So what this means is that when we evaluate the polynomial at r, we must do it in the unencrypted domain and THEN
# encrypt that value. Also, when we generate the powers of r, we must do it in then unencrypted domain and THEN encrypt them.


In [67]:

# This is where the prover class is implemented and it will be using Protocol and named
# using the chapter number defining the protocol in the PDF
import numpy as np

class Prover(ABC):
    def __init__(self, powers_of_s: list[int], target_polynomial_coefficients: list[int], p_polynomial_coefficients: list[int], field_prime:int = field_prime, generator:int = generator):
        super().__init__()
        self.field_prime = field_prime
        #self.GF = galois.GF(field_prime)
        self.encrypted_powers = powers_of_s
        self.generator = generator
        self.target_polynomial_coefficients = target_polynomial_coefficients
        self.p_polynomial_coefficients = p_polynomial_coefficients
        self.target_desc_coeffs = list(reversed(target_polynomial_coefficients))
    
    @abstractmethod
    def calculate_h(self):
        ...

    @abstractmethod
    def evaluate_encrypted_h(self):
        ...
    

class Prover_334(Prover):
# The prover's tasks are as follows:
# (a) Calculate polynomial h(x) = p(x)/t(x) 
# (b) Using encrypted powers g^s^0, g^s^1, ... , g^s^d and coefficients c0, c1, ... , cn 
# evaluates E (p(s)) = g^p(s) = (g^s^d)^cd ··· (g^s^1)^c1 · ( g^s^0 )^c0 
# (c) and similarly E (h(s)) = g^h(s) 
# (d) The resulting g^p and g^h are provided to the verifier


    def __init__(self, powers_of_s: list[int], target_polynomial_coefficients: list[int], p_polynomial_coefficients: list[int], field_prime:int = field_prime, generator:int = generator):
        super().__init__(powers_of_s, target_polynomial_coefficients, p_polynomial_coefficients, field_prime, generator)

    
    def calculate_h(self):
        # (a)
        calculated_h = np.polynomial.polynomial.polydiv(self.p_polynomial_coefficients,self.target_polynomial_coefficients)
        if calculated_h[1] != 0:
            raise ValueError('polynomial is not divisible')
        else:
            self.h_polynomial_coefficients = list(map(int, calculated_h[0].astype(int)))
        print(f'h polynomial is {self.h_polynomial_coefficients}')

    def evaluate_encrypted_p(self):
        #print(galois.primitive_root(self.field_prime))
        # so what I need to do is array multiplication between encrypted powers and p_polynomial_coefficients
        # in the GF. Is that possible? Well it is, but it returns non modulo values, which is a problem.
        result = 1
        for index, coeff in enumerate(self.p_polynomial_coefficients):
            result = (result * pow(self.encrypted_powers[index],int(coeff),self.field_prime)) % self.field_prime
        self.encrypted_p = result


    def evaluate_encrypted_h(self):
        # do the same thing as with p(x), but shrink encrypted powers to right size
        result = 1
        for index, coeff in enumerate(self.h_polynomial_coefficients):
            result = (result * pow(self.encrypted_powers[index],int(coeff),self.field_prime)) % self.field_prime
        self.encrypted_h = result
# let's exercise this new class
# target poly is (x-1)(x-2), which is equivalent to 2 -3x +x^2
# we now want to see what happens with p(x) = (x-1)(x-2)(x-3), which is x^3 - 6 x^2 + 11 x - 6
# print(f'Encrypted powers are: {myVerifier.encrypted_powers}')
# myProver =  Prover_334(myVerifier.encrypted_powers, [2,-3,1],[-6,11,-6,1])
# print(f'p(x) = {myProver.p_polynomial_coefficients}')
# myProver.calculate_h()
# myProver.evaluate_encrypted_p()
# myProver.evaluate_encrypted_h()


# print(f'The E(P(s)) is {myProver.encrypted_p}, my E(H(s)) is {myProver.encrypted_h}')
# encrypted_p = myProver.encrypted_p
# encrypted_h = myProver.encrypted_h
# print(f'T(s) = {myVerifier.evaluated_poly}')
# print(f'the answer is : {pow(encrypted_h,myVerifier.evaluated_poly,field_prime)}')

In [68]:
# Now let's use the 3.3.4 verifier and prover classes with another bigger sized example
# The P(x) will be (x - 1) (x - 2) (x - 3) (x - 4)
# The target polynomial T(x) will be (x - 1) (x - 2) (x - 3)
# The P(x)/T(x) = H(x) will be (x-4)

t_poly_coeffs = calculate_coeff_from_roots([1,2,3])
p_poly_coeffs = calculate_coeff_from_roots([1,2,3,4])
myVerifier = Verifier_334(t_poly_coeffs,5)
myVerifier.calculate_encrypted_powers()
myVerifier.evaluate_unencrypted_target_polynomial()
myProver = Prover_334(myVerifier.encrypted_powers,t_poly_coeffs,p_poly_coeffs)
myProver.calculate_h()
myProver.evaluate_encrypted_h()
myProver.evaluate_encrypted_p()
if myVerifier.check_encrypted_t_h_p(myProver.encrypted_h,myProver.encrypted_p):
    print(f'Everything checks out')

the polynomial evaluated at s = 2400 is 2550.0 
h polynomial is [-4, 1]
Everything checks out


In [69]:
from typing import Protocol
from abc import ABC, abstractmethod
class TestParent(ABC):
    def __init__(self, param_a:int, param_b:int):
        super().__init__()
        self.a = param_a
        self.b = param_b

class TestChild(TestParent):
    def __init__(self, param_a, param_b):
        super().__init__(param_a, param_b)
        self.c = self.a + 100

myTest = TestChild(4,5)
    

The next step is to restrict the usage of the polynomial.
What do we mean by this?

Well, we can see that the protocol implemented from chapter 3.3.4 has a major weakness, which is that the prover does NOT have to use the encrypted powers, or even a polynomial to provide a valid answer.
One could use any possible means to find some arbitrary values $z^p$ and $z^h$ which satisfy equation $z^p = (zh)^{t(s)}$ and provide them to the verifier instead of $g^p$ and $g^h$. For example, for some random r, $zh = g^r$ and $zp = (g^{t(s)})^r$, where $gt(s)$ can be computed from the provided encrypted powers of s. That is why the verifier needs the proof that only supplied encryptions of powers of s were used to calculate gp and gh and nothing else.

So, we are going to force the prover to use the encrypted powers by having him perform the same calculations twice, but with encrypted powers $\alpha$ shifted. Let's explore this concept to understand it and see how it applies here.

Alice chooses a random value $\alpha$, which she uses to exponentiate a value x that she requires Bob to exponentiate. So, she generates $\alpha$ (let's say 3) and wants x (let's say it's 2) to be exponentiated. She generates $x' = x^{\alpha}$ and gives x and x' to Bob. He does not know what $\alpha$ is. For this example, she gives Bob 2 and $2^3 \mod 7 = 1$

Now Bob exponentiate x and x' to the power of z = 5 (his choice) in the prime field p=7. So he computes $y = x^z \mod p$ and $y' = x'^z \mod p$. $y = 2^5 \mod 7 = 4$ and $y' = 1^6 \mod 7 = 1$.

Now Alice receives 4 and 1, and can verify that $y^{\alpha} = y' (\mod p)$, that is $4^3 (\mod 7) = 1$

What Alice has been able to prove is that $y^{\alpha} = y' (\mod p) \equiv (x^z)^{\alpha} = x'^z (\mod p) \equiv x^{z * \alpha} = (x^{\alpha})^z $

Note that with such small numbers, it is possible for Bob to figure out what $\alpha$ was chosen, but with real size cases we are looking at hundreds of bits in size for each of these numbers, which makes the brute force all but impossible.

Now what we have just demonstrated is that Alice (the verifier), can ensure that Bob (the prover) has in fact used the encrypted powers she provided to him by requiring him to perform the coefficents multiplication with both the alpha-shifted version and the plain version of the encrypted powers.
For the same reasons that Bob cannot find alpha, Alice cannot figure out which coefficients (5 in our example) Bob used to exponentiate the x and x' values. This is good, because we want the prover to be able to keep the coefficients of the polynomial secret.

Now this works for one order of the polyonmial, but what about making it work for the whole thing?
This is actually pretty easy. We just apply that technique to the whole polynomial, as the sum of each order in the encrypted domain does not disrupt the alpha shift. The prover generates $g^p$, $g^{p'}$ and $g^h$.

In [70]:
# Let's implement the protocol described in chapter 3.4

class Verifier_34(Verifier):
    # (a) Verifier chooses random s, α 
    # (b) and provides encrypted powers g^s^0, g^s^1, . . . , g^s^d and their shifts g^αs^0, g^αs^1, . . . , g^αs^d
    # (c) Prover evaluates encrypted polynomial with provided powers of s:
    #   g^p(s) =(g^s^0 )^c0·(g^s^1 )^c1· . . . ·(g^s^d )^cd = g^(c0s^0+c1s^1+...+cds^d)
    # (d) Prover evaluates encrypted “shifted” polynomial with the corresponding α-shifts of the powers of s:
    #   g^αp(s) =(g^αs^0 )^c0·(g^αs^1 )^c1· . . . ·(g^αs^d )^cd = g^c0αs^0+c1αs^1+...+cdαs^d = g^α(c0s0+c1s1+...+cdsd)
    # (e) Prover provides the result as g^p, g^p′ to the verifier
    # (f) Verifier checks: (g^p)α = g^p' AND as with former version of protocol, p = t(s) · h in encrypted space: 
    # g^p =(g^h)^t(s) ⇒ g^p = g^t(s)·h 
    def __init__(self, target_polynomial_coefficients: list[int], max_order_p_poly:int, field_prime:int = field_prime, generator:int = generator):
        super().__init__(target_polynomial_coefficients, max_order_p_poly, field_prime, generator)
        self.s = secrets.randbelow(self.field_prime) # (a)
        self.alpha = secrets.randbelow(self.field_prime) # (a)
        self.alpha_shifted_encrypted_powers = []
        self.encrypted_p = None
        self.encrypted_p_prime = None
        #print(f'secret = {self.s}')

    def calculate_encrypted_powers(self): #(b)
        # calculate encrypted powers and alpha-shifted encrypted powers
        for index in range(self.max_order_p_poly):
            self.encrypted_powers.append(encrypt(pow(self.s,index)))
            self.alpha_shifted_encrypted_powers.append(encrypt(self.alpha*pow(self.s,index)))
        print(f'Secret value S is: {self.s}')
        print(f'Powers of S: {self.encrypted_powers}')
        print(f'Shifted Powers of S: {self.alpha_shifted_encrypted_powers}')
        other_way_to_shift = [ pow(power,self.alpha,self.field_prime) for power in self.encrypted_powers ]
        print(f'Other way to compute shifts : {other_way_to_shift}')

    def evaluate_unencrypted_target_polynomial(self):
        result=0
        for index,coeff in enumerate(self.target_polynomial_coefficients):
            result = result + coeff*pow(self.s,index)
        self.evaluated_poly = result
        
        # The numpy implementation is buggy, it returns results that are wrong when numbers are really big
        # This is why I replaced it with the above, which uses the Python integer facility with large numbers natively and correctly
        #self.evaluated_poly = int(np.polynomial.polynomial.polyval(self.s, self.target_polynomial_coefficients))
        # print(f'The target polynomial is: {self.target_desc_coeffs}')
        # print(f'the polynomial evaluated at s = {self.s} is {self.evaluated_poly} ')

    def set_secrets(self, s:int, alpha:int):
        self.s = s
        self.alpha = alpha

    def check_encrypted_t_h_p_alpha(self, encrypted_h:int, encrypted_p:int, encrypted_p_prime:int) -> bool:
        # (f)
        print(f'Encrypted p is: {encrypted_p}, encrypted p\' is : {encrypted_p_prime} and alpha is :{self.alpha}')
        print(f'Encrypted p ^alpha mod field_prime is :{pow(encrypted_p,self.alpha,self.field_prime)}')
        print(f'Encrypted H is: {encrypted_h}')
        print(f'Encrypted h ^t(s) mod field prime is : {pow(encrypted_h,self.evaluated_poly,self.field_prime)}') 
        return (pow(encrypted_h,self.evaluated_poly,self.field_prime) == encrypted_p) and \
            encrypted_p_prime == pow(encrypted_p,self.alpha,self.field_prime)


class Prover_34(Prover):
    def __init__(self, powers_of_s: list[int], shifted_powers_of_s: list[int], target_polynomial_coefficients: list[int], p_polynomial_coefficients: list[int], field_prime:int = field_prime, generator:int = generator):
        super().__init__(powers_of_s, target_polynomial_coefficients, p_polynomial_coefficients, field_prime, generator)
        self.shifted_powers_of_s = shifted_powers_of_s
    
    def calculate_h(self):
        # (a)
        calculated_h = np.polynomial.polynomial.polydiv(self.p_polynomial_coefficients,self.target_polynomial_coefficients)
        if calculated_h[1] != 0:
            raise ValueError('polynomial is not divisible')
        else:
            self.h_polynomial_coefficients = list(map(int, calculated_h[0].astype(int)))
        print(f'h polynomial is {self.h_polynomial_coefficients}')

    def evaluate_encrypted_p_and_p_prime(self):
        # evaluate p and p' (with shifted encrypted powers) in the encrypted domain
        enc_p = 1
        enc_p_prime = 1
        for index, coeff in enumerate(self.p_polynomial_coefficients):
            enc_p = (enc_p * pow(self.encrypted_powers[index],int(coeff),self.field_prime)) % self.field_prime
            enc_p_prime = (enc_p_prime * pow(self.shifted_powers_of_s[index],int(coeff),self.field_prime)) % self.field_prime
        self.encrypted_p = enc_p
        self.encrypted_p_prime = enc_p_prime

    def evaluate_encrypted_p_and_p_prime_debug(self, alpha:int):
        # evaluate p and p' (with shifted encrypted powers) in the encrypted domain
        # this is used to debug and see what happens for each order of the polynomial and when it breaks down
        # the alpha used by the verifier needs to be provided to check each step
        enc_p = 1
        enc_p_prime = 1
        for index, coeff in enumerate(self.p_polynomial_coefficients):
            enc_p = (enc_p * pow(self.encrypted_powers[index],int(coeff),self.field_prime)) % self.field_prime
            enc_p_prime = (enc_p_prime * pow(self.shifted_powers_of_s[index],int(coeff),self.field_prime)) % self.field_prime
            print(f'encrypted p at order {index} is:{enc_p}')
            print(f'encrypted p\' is {enc_p_prime}')
            print(f'Encrypted p^alpha is: {pow(enc_p,alpha,self.field_prime)}')
        self.encrypted_p = enc_p
        self.encrypted_p_prime = enc_p_prime

    def evaluate_encrypted_h(self):
        # do the same thing as with p(x), but shrink encrypted powers to right size
        result = 1
        for index, coeff in enumerate(self.h_polynomial_coefficients):
            result = (result * pow(self.encrypted_powers[index],int(coeff),self.field_prime)) % self.field_prime
        self.encrypted_h = result

In [71]:
# Let's test the 3.4 protocol
t_poly_coeffs = calculate_coeff_from_roots([1,2,3,4,5])
p_poly_coeffs = calculate_coeff_from_roots([1,2,3,4,5,6,7])
myVerifier = Verifier_34(t_poly_coeffs,8)
# for debugging
# myVerifier.set_secrets(165,2)

myVerifier.calculate_encrypted_powers()
myVerifier.evaluate_unencrypted_target_polynomial()
myProver = Prover_34(myVerifier.encrypted_powers, myVerifier.alpha_shifted_encrypted_powers, t_poly_coeffs,p_poly_coeffs)
myProver.calculate_h()
myProver.evaluate_encrypted_h()
myProver.evaluate_encrypted_p_and_p_prime()
if myVerifier.check_encrypted_t_h_p_alpha(myProver.encrypted_h,myProver.encrypted_p,myProver.encrypted_p_prime):
    print(f'Everything checks out')

# for debugging
# myProver.evaluate_encrypted_p_and_p_prime_debug(2)

Secret value S is: 6772
Powers of S: [7, 4420, 1437, 6764, 5858, 706, 4531, 981]
Shifted Powers of S: [4392, 3228, 2950, 5242, 5449, 4846, 6127, 7664]
Other way to compute shifts : [4392, 3228, 2950, 5242, 5449, 4846, 6127, 7664]
h polynomial is [42, -13, 1]
Encrypted p is: 6315, encrypted p' is : 4859 and alpha is :5155
Encrypted p ^alpha mod field_prime is :4859
Encrypted H is: 7092
Encrypted h ^t(s) mod field prime is : 6315
Everything checks out


Now that we have a way to protect the verifier's secret value and force the prover to use the powers provided to him, we will also make sure to protect the prover's knowledge of the coefficients of $p(x)$. This is partly where the zero knowledge moniker comes from.

The way to do this is very similar to the way we enforced the usage of the powers of S, namely by applying a shift to the powers. This time we shift by the $\delta$ power on both sides of the equations that are validated.

Let's recap what the verifier does to make sure the prover knows the polynomial.
The prover provides the following $g^p$, $g^{p′}$, $g^h$

The Verifier then checks :

$g^p =(g^h)^{t(s)}$         (polynomial p(x) has roots of t(x))

$(g^p)^α = g^{p′}$               (polynomial of a correct form is used)

So now the Verifier randomly chooses a $\delta$ and exponentiates his proof values with it such as $(g^{p(s)})^δ$, $(g^{h(s)})^δ$ , $(g^{αp(s)})^δ$ :

$(g^p)^{\delta} =((g^h)^{\delta})^{t(s)}$         (polynomial p(x) has roots of t(x))

$((g^p)^{\delta})^α = (g^{p′})^{\delta}$               (polynomial of a correct form is used)

These equations are consolidated the following way:

$g^{p * \delta} =g^{h * \delta * t(s)}$

$g^{p * \delta * α} = g^{p′* \delta}$

So that is all we need to implement for the chapter 3.5 protocol, and we basically do not need to change anything in the protocol of the verifier. So only the prover is getting an update.


In [72]:
# Let's implement the protocol described in chapter 3.4

class Verifier_35(Verifier_34):
    # (a) Verifier chooses random s, α 
    # (b) and provides encrypted powers g^s^0, g^s^1, . . . , g^s^d and their shifts g^αs^0, g^αs^1, . . . , g^αs^d
    # (c) Prover evaluates encrypted polynomial with provided powers of s:
    #   g^p(s) =(g^s^0 )^c0·(g^s^1 )^c1· . . . ·(g^s^d )^cd = g^(c0s^0+c1s^1+...+cds^d)
    # (d) Prover evaluates encrypted “shifted” polynomial with the corresponding α-shifts of the powers of s:
    #   g^αp(s) =(g^αs^0 )^c0·(g^αs^1 )^c1· . . . ·(g^αs^d )^cd = g^c0αs^0+c1αs^1+...+cdαs^d = g^α(c0s0+c1s1+...+cdsd)
    # (e) Prover provides the result as g^p, g^p′ to the verifier
    # (f) Verifier checks: (g^p)α = g^p' AND as with former version of protocol, p = t(s) · h in encrypted space: 
    # g^p =(g^h)^t(s) ⇒ g^p = g^t(s)·h 
    def __init__(self, target_polynomial_coefficients: list[int], max_order_p_poly:int, field_prime:int = field_prime, generator:int = generator):
        super().__init__(target_polynomial_coefficients, max_order_p_poly, field_prime, generator)


class Prover_35(Prover):
    # The prover now has to generate a random number delta to exponentiate the results
    def __init__(self, powers_of_s: list[int], shifted_powers_of_s: list[int], target_polynomial_coefficients: list[int], p_polynomial_coefficients: list[int], field_prime:int = field_prime, generator:int = generator):
        super().__init__(powers_of_s, target_polynomial_coefficients, p_polynomial_coefficients, field_prime, generator)
        self.shifted_powers_of_s = shifted_powers_of_s
        self.delta = secrets.randbelow(self.field_prime)
    
    def calculate_h(self):
        # (a)
        calculated_h = np.polynomial.polynomial.polydiv(self.p_polynomial_coefficients,self.target_polynomial_coefficients)
        if calculated_h[1] != 0:
            raise ValueError('polynomial is not divisible')
        else:
            self.h_polynomial_coefficients = list(map(int, calculated_h[0].astype(int)))
        print(f'h polynomial is {self.h_polynomial_coefficients}')

    def evaluate_encrypted_p_and_p_prime(self):
        # evaluate p and p' (with shifted encrypted powers) in the encrypted domain
        enc_p = 1
        enc_p_prime = 1
        for index, coeff in enumerate(self.p_polynomial_coefficients):
            enc_p = (enc_p * pow(self.encrypted_powers[index],int(coeff),self.field_prime)) % self.field_prime
            enc_p_prime = (enc_p_prime * pow(self.shifted_powers_of_s[index],int(coeff),self.field_prime)) % self.field_prime
        self.encrypted_p = enc_p
        self.encrypted_p_prime = enc_p_prime

    def evaluate_encrypted_p_and_p_prime_debug(self, alpha:int):
        # evaluate p and p' (with shifted encrypted powers) in the encrypted domain
        # this is used to debug and see what happens for each order of the polynomial and when it breaks down
        # the alpha used by the verifier needs to be provided to check each step
        enc_p = 1
        enc_p_prime = 1
        for index, coeff in enumerate(self.p_polynomial_coefficients):
            enc_p = (enc_p * pow(self.encrypted_powers[index],int(coeff),self.field_prime)) % self.field_prime
            enc_p_prime = (enc_p_prime * pow(self.shifted_powers_of_s[index],int(coeff),self.field_prime)) % self.field_prime
            print(f'encrypted p at order {index} is:{enc_p}')
            print(f'encrypted p\' is {enc_p_prime}')
            print(f'Encrypted p^alpha is: {pow(enc_p,alpha,self.field_prime)}')
        self.encrypted_p = enc_p
        self.encrypted_p_prime = enc_p_prime

    def evaluate_encrypted_h(self):
        # do the same thing as with p(x), but shrink encrypted powers to right size
        result = 1
        for index, coeff in enumerate(self.h_polynomial_coefficients):
            result = (result * pow(self.encrypted_powers[index],int(coeff),self.field_prime)) % self.field_prime
        self.encrypted_h = result

    def delta_shift(self):
        self.delta_shifted_encrypted_p = pow(self.encrypted_p,self.delta,self.field_prime)
        self.delta_shifted_encrypted_p_prime = pow(self.encrypted_p_prime,self.delta,self.field_prime)
        self.delta_shifted_encrypted_h = pow(self.encrypted_h,self.delta,self.field_prime)
        print(f'The delta value is :{self.delta}')
        print(f'delta shifted encrypted p: {self.delta_shifted_encrypted_p}')
        print(f'delta shifted encrypted p prime : {self.delta_shifted_encrypted_p_prime}')
        print(f'delta shifted encrypted h: {self.delta_shifted_encrypted_h}')

In [73]:
# Let's test the 3.5 protocol
t_poly_coeffs = calculate_coeff_from_roots([1,2,3,4,5])
p_poly_coeffs = calculate_coeff_from_roots([1,2,3,4,5,6,7])
myVerifier = Verifier_35(t_poly_coeffs,8)
# for debugging
# myVerifier.set_secrets(165,2)

myVerifier.calculate_encrypted_powers()
myVerifier.evaluate_unencrypted_target_polynomial()
myProver = Prover_35(myVerifier.encrypted_powers, myVerifier.alpha_shifted_encrypted_powers, t_poly_coeffs,p_poly_coeffs)
myProver.calculate_h()
myProver.evaluate_encrypted_h()
myProver.evaluate_encrypted_p_and_p_prime()
myProver.delta_shift()
if myVerifier.check_encrypted_t_h_p_alpha(myProver.delta_shifted_encrypted_h,myProver.delta_shifted_encrypted_p,myProver.delta_shifted_encrypted_p_prime):
    print(f'Everything checks out')

# for debugging
# myProver.evaluate_encrypted_p_and_p_prime_debug(2)


Secret value S is: 786
Powers of S: [7, 3378, 7247, 5956, 5768, 760, 1682, 835]
Shifted Powers of S: [283, 6878, 2069, 2241, 4511, 4656, 2869, 5002]
Other way to compute shifts : [283, 6878, 2069, 2241, 4511, 4656, 2869, 5002]
h polynomial is [42, -13, 1]
The delta value is :5651
delta shifted encrypted p: 3887
delta shifted encrypted p prime : 4621
delta shifted encrypted h: 6946
Encrypted p is: 3887, encrypted p' is : 4621 and alpha is :6307
Encrypted p ^alpha mod field_prime is :4621
Encrypted H is: 6946
Encrypted h ^t(s) mod field prime is : 3887
Everything checks out


The next step to implement the ZK-SNARK protocol is to get rid of the interactivity that we still have, by having to go back and forth between the prover and verifier. This removes weaknesses of the protocol that allow:
- The prover and verifier to collude (e.g. the verifier could simply give alpha and s to the prover and thus allow the prover to forge proofs)
- The verifier itself can generate fake proofs
- Attack surface to be bigger on the verifier because it has to store alpha and t(s) until the verification step is performed

The current design also makes it so it only works for a dedicated verifier at a time, it requires the prover to be online at all times if we want to have more than one verifier verifying a proof, so overall it is not practical for systems that need to be permanent and/or that need to be available to any verifier. Hence, we need the secret parameters to be reusable, public, trustworthy and infeasible to abuse.

First, let's see how we can secure the verifier's secrets that are necessary to verify the proof: t(s) and alpha. It we can encrypt them and provide them to the prover somehow, then we would get rid of the requirement of having the verifier generate them and store them... maybe but at least if we send them to the prover encrypted, then he could do the verification by multiplying encrypted values of t(s) and h as well as p and α.. That is except for the fact that in the encrypted domain it is not possible to multiply two encrypted values... so what gives?

There is something called cryptographic pairings that can help us with this issue. Let's take a look at it from a high-level. This [article](https://medium.com/@VitalikButerin/exploring-elliptic-curve-pairings-c73c1864e627) from Vitalik is a good primer for pairings.

Now before going through this, I think replacing the encryption function with ECC should come first and see if we can replicate the success of protocol 3.5 before jumping into the mess of pairings.


The way we encrypt a number with ECC is by choosing a curve and choosing a generator point in it. For instance we can chose G = (1,3), but in reality known curves use known points for their generator points. The [library](https://py-ecc.readthedocs.io/en/latest/index.html) we will be using was written in Python by the Ethereum Foundation and for the curve $y^2=x^3 +3$ and has the following characteristics:
### Curve order
curve_order = (
    21888242871839275222246405745257275088548364400416034343698204186575808495617
)

### Field properties
"bn128": {
    "field_modulus": 21888242871839275222246405745257275088696311157297823662689037894645226208583,  # noqa: E501
    "fq2_modulus_coeffs": (1, 0),
    "fq12_modulus_coeffs": (82, 0, 0, 0, 0, 0, -18, 0, 0, 0, 0, 0),  # Implied + [1]
},
### Curve is $y^2=x^3 +3$
b = FQ(3)
### Twisted curve over $FQ^2$
b2 = FQ2([3, 0]) / FQ2([9, 1])
### Extension curve over $FQ^{12}$; same b value as over FQ
b12 = FQ12([3] + [0] * 11)

### Generator for curve over FQ
G1 = (FQ(1), FQ(2))
### Generator for twisted curve over FQ2
G2 = (
    FQ2(
        [
            10857046999023057135944570762232829481370756359578518086990519993285655852781,  # noqa: E501
            11559732032986387107991004021392285783925812861821192530917403151452391805634,  # noqa: E501
        ]
    ),
    FQ2(
        [
            8495653923123431417604973247489272438418190587263600148770280649306958101930,  # noqa: E501
            4082367875863433681332203403145435568316851327593401208105741076214120093531,  # noqa: E501
        ]
    ),
)
### Point at infinity over FQ
Z1 = None
### Point at infinity for twisted curve over FQ2
Z2 = None

This looks all very complicated but for now all we'll need is to use G1 over FQ to encrypt our values by multiplying them by G1 and then add the encrypted values as is warranted.

With the former scheme of homomorphic encryption, it was a little difficult at times to know if we were applying operations in the encrypted domain or in the clear domain, but here it is really quite clear. Then encrypted domain is always in the form of a point [x,y].

In order to understand how things are implemented looking at the [Github page](https://github.com/ethereum/py_ecc/blob/main/tests/core/test_bn128_and_bls12_381.py) for the tests of the library is very useful.


In [74]:
# Let's play with ECC for homomorphic encryption and see if it actually works like it should
from py_ecc import (
    bn128,
    optimized_bn128,
    bls12_381
)
from py_ecc.fields import (
    bn128_FQ,
    bn128_FQ2,
    bn128_FQ12,
)
from py_ecc.fields.field_properties import (
    field_properties,
)

curve = 'bn128'

if curve == 'optimized_bn128':    
    G1 = optimized_bn128.G1
    G2 = optimized_bn128.G2
    G12 = optimized_bn128.G12
    pairing = optimized_bn128.pairing
    neg = optimized_bn128.neg
    multiply = optimized_bn128.multiply
    eq = optimized_bn128.eq
    curve_order = optimized_bn128.curve_order
    add = optimized_bn128.add
    field_modulus = field_properties["bn128"]["field_modulus"]
elif curve == 'bn128':
    G1 = bn128.G1
    G2 = bn128.G2
    G12 = bn128.G12
    pairing = bn128.pairing
    neg = bn128.neg
    multiply = bn128.multiply
    eq = bn128.eq
    curve_order = bn128.curve_order
    add = bn128.add
    field_modulus = field_properties["bn128"]["field_modulus"]
elif curve == 'bls12_381':
    G1 = bls12_381.G1
    G2 = bls12_381.G2
    G1 = bls12_381.G12
    pairing = bls12_381.pairing
    neg = bls12_381.neg
    multiply = bls12_381.multiply
    eq = bls12_381.eq
    curve_order = bls12_381.curve_order
    add = bls12_381.add
    field_modulus = field_properties["bls12_381"]["field_modulus"]

def encrypt_over_FQ(value: int):

    if value < 0:
        return multiply(G1, curve_order + value)
    else:
        return multiply(G1,value)
    
def encrypt_over_FQ2(value: int):
    if value < 0:
        return multiply(G2, curve_order + value)
    else:
        return multiply(G2,value)

print(f'Encrypt 1 see what that is: {encrypt_over_FQ(1)}')
print(f'Encrypt 2 see what that is: {encrypt_over_FQ(2)}')
print(f'From this we can see that the G1 point is at (1,2) and as soon as we multiply it by 2, it goes to really large values')

# let's see if we can perform operations in the encrypted domain
# To start we want to be able to compute for instance 2x + 3x^2, where we provide encrypted powers
# so encrypted x and x^2, for a given value of x, s = 3 => s^1 = 3, s^2=9
# the result in the clear is 2 * 3 + 3 * 9 = 33
s1 = encrypt_over_FQ(3)
s2 = encrypt_over_FQ(9)

enc_result = add(multiply(s1,2),multiply(s2,3))
print(f'Is it the same? {eq(enc_result,encrypt_over_FQ(33))}')

# now how do we deal with the alpha-shifted (and delta-shifted) values when using this?
# For the alpha-shift, the reason it works with the multiplicative group scheme is that
# the encrypted multiplication is actually an exponentation.
# With this scheme, we cannot exponentiate a point in FQ, but the equivalent operation is a multiplication.
# It is still impossible for the Prover to figure out what alpha or s was, as it's impossible to go from nG1 to n
# With alpha = 2, and same values as above, so 2*(s^1) = 6, 2*(s^2) = 18
# the shifted result in the clear is 2 * 6 + 3 * 18 = 66

alpha = 2
s1_alpha = encrypt_over_FQ(6)
s2_alpha = encrypt_over_FQ(18)
enc_shifted_result = add(multiply(s1_alpha,2),multiply(s2_alpha,3))
print(f'Is it the same? {eq(enc_shifted_result,encrypt_over_FQ(66))}')

print(f'And now we see if the scheme works by multiplying enc_result by alpha to see if we get back enc_shifted_result')
print(f'Does it work? {eq(multiply(enc_result,alpha),enc_shifted_result)}')


# The same thing is done with the delta-shift by the prover

# Now let's see how to use the zero point, or point at infinity. 
# The way it works is by having it set to None, but it only works for non-optimized curves
# for the BLS curves it's different

zero = None

print(f'Is it the G1 point? {add(zero,G1)}')
assert(eq(add(zero,G1),G1))

# let's see how that stuff works with negative numbers, let's see if s =3, coeff = -1 see if encrypted(-3) works
s1 = encrypt_over_FQ(3)
# enc_result = bn128.multiply(s1,-1)
# the above does not work, we need to use bn128.curve_order - 1
# This means the encrypt_ecc function needs to handle the negative numbers too
enc_result = multiply(s1,curve_order - 1)
print(enc_result)

assert(eq(enc_result,encrypt_over_FQ(curve_order-3)))

# let's see if combination of negative and positives work
# f(x)=2x - 3x^2, with s=3, s^2=9 => 6 - 27 = -21
s2=encrypt_over_FQ(9)
enc_result = add(multiply(s1,2),multiply(s2,curve_order-3))
assert(eq(enc_result,encrypt_over_FQ(-21)))

# can we instaed always use curve_order + value?
print(f'This is supposed to be G1 : {multiply(s2,curve_order + 1)}')
# this does not work


Encrypt 1 see what that is: (1, 2)
Encrypt 2 see what that is: (1368015179489954701390400359078579693043519447331113978918064868415326638035, 9918110051302171585080402603319702774565515993150576347155970296011118125764)
From this we can see that the G1 point is at (1,2) and as soon as we multiply it by 2, it goes to really large values
Is it the same? True
Is it the same? True
And now we see if the scheme works by multiplying enc_result by alpha to see if we get back enc_shifted_result
Does it work? True
Is it the G1 point? (1, 2)
(3353031288059533942658390886683067124040920775575537747144343083137631628272, 2566709105286906361299853307776759647279481117519912024775619069693558446822)
This is supposed to be G1 : (1624070059937464756887933993293429854168590106605707304006200119738501412969, 3269329550605213075043232856820720631601935657990457502777101397807070461336)


In [75]:
#Now let's implement the protocol 3.6.0, which is the same as 3.5 but using ECC for homomorphic encryption 
# instead of a multiplicative group.
class Verifier_360(Verifier):
    # The changes needed to make it work with a single ECC field are as follows:
    # Encrypted multiplication is done by multiplying the G1 (generator) point by an integer.
    # If the integer is negative, then we need to multiply by bn128.curve_order + negative value
    # Encrypted addition is simply addition of 2 ECC points
    # The zero point is actually None
    # The field_prime must be set to that of the curve's characteristics
    # The alpha and delta shifts are achieved via multiplication of the values to be shifted instead of exponetating them
    # The reason is simple, with this homomorphic scheme, we multiply the points, we do not exponentiate
    
    def __init__(self, target_polynomial_coefficients: list[int], max_order_p_poly:int, field_prime:int = field_prime, generator:int = generator):
        super().__init__(target_polynomial_coefficients, max_order_p_poly, field_prime, generator)
        self.s = secrets.randbelow(field_prime) # (a)
        self.alpha = secrets.randbelow(field_prime) # (a)
        self.alpha_shifted_encrypted_powers = []
        self.encrypted_p = None
        self.encrypted_p_prime = None
        #print(f'secret = {self.s}')

    def calculate_encrypted_powers(self): #(b)
        # calculate encrypted powers and alpha-shifted encrypted powers
        for index in range(self.max_order_p_poly):
            self.encrypted_powers.append(encrypt_over_FQ(pow(self.s,index)))
            self.alpha_shifted_encrypted_powers.append(encrypt_over_FQ(self.alpha*pow(self.s,index)))
        print(f'Secret value S is: {self.s}')
        print(f'Powers of S: {self.encrypted_powers}')
        print(f'Shifted Powers of S: {self.alpha_shifted_encrypted_powers}')
        #other_way_to_shift = [ pow(power,self.alpha,self.field_prime) for power in self.encrypted_powers ]
        #print(f'Other way to compute shifts : {other_way_to_shift}')

    def evaluate_unencrypted_target_polynomial(self):
        result=0
        for index,coeff in enumerate(self.target_polynomial_coefficients):
            result = result + coeff*pow(self.s,index)
        self.evaluated_poly = result
        
        # The numpy implementation is buggy, it returns results that are wrong when numbers are really big
        # This is why I replaced it with the above, which uses the Python integer facility with large numbers natively and correctly
        #self.evaluated_poly = int(np.polynomial.polynomial.polyval(self.s, self.target_polynomial_coefficients))
        # print(f'The target polynomial is: {self.target_desc_coeffs}')
        # print(f'the polynomial evaluated at s = {self.s} is {self.evaluated_poly} ')

    def set_secrets(self, s:int, alpha:int):
        self.s = s
        self.alpha = alpha

    def check_encrypted_t_h_p_alpha(self, encrypted_h:bn128_FQ, encrypted_p:bn128_FQ, encrypted_p_prime:bn128_FQ) -> bool:
        # (f)
        print(f'Encrypted p is: {encrypted_p}, encrypted p\' is : {encrypted_p_prime} and alpha is :{self.alpha}')
        print(f'Encrypted p * alpha is :{multiply(encrypted_p,self.alpha)}')
        print(f'Encrypted H is: {encrypted_h}')
        encrypted_h_times_t = bn128.multiply(encrypted_h,curve_order + self.evaluated_poly) if (self.evaluated_poly < 0) else multiply(encrypted_h,self.evaluated_poly)
        print(f'Encrypted h * t(s) is : {encrypted_h_times_t}') 
        return (eq(encrypted_h_times_t,encrypted_p) and \
            eq(encrypted_p_prime, multiply(encrypted_p,self.alpha)))
    

class Prover_360(Prover):
    # The prover now has to generate a random number delta to exponentiate the results
    def __init__(self, powers_of_s: list[int], shifted_powers_of_s: list[int], target_polynomial_coefficients: list[int], p_polynomial_coefficients: list[int], field_prime:int = field_prime, generator:int = generator):
        super().__init__(powers_of_s, target_polynomial_coefficients, p_polynomial_coefficients, field_prime, generator)
        self.shifted_powers_of_s = shifted_powers_of_s
        self.delta = secrets.randbelow(self.field_prime)
    
    def calculate_h(self):
        # (a)
        calculated_h = np.polynomial.polynomial.polydiv(self.p_polynomial_coefficients,self.target_polynomial_coefficients)
        if calculated_h[1] != 0:
            raise ValueError('polynomial is not divisible')
        else:
            self.h_polynomial_coefficients = list(map(int, calculated_h[0].astype(int)))
        print(f'h polynomial is {self.h_polynomial_coefficients}')

    def evaluate_encrypted_p_and_p_prime(self):
        # evaluate p and p' (with shifted encrypted powers) in the encrypted domain
        enc_p = None
        enc_p_prime = None
        for index, coeff in enumerate(self.p_polynomial_coefficients):
            if coeff >= 0:
                enc_p = add(enc_p, multiply(self.encrypted_powers[index],int(coeff)))
                enc_p_prime = bn128.add(enc_p_prime, bn128.multiply(self.shifted_powers_of_s[index], int(coeff)))
            else:
                enc_p = add(enc_p, multiply(self.encrypted_powers[index],curve_order + int(coeff)))
                enc_p_prime = add(enc_p_prime, multiply(self.shifted_powers_of_s[index], curve_order + int(coeff)))          
        self.encrypted_p = enc_p
        self.encrypted_p_prime = enc_p_prime

    def evaluate_encrypted_h(self):
        # do the same thing as with p(x), but shrink encrypted powers to right size
        result = None
        for index, coeff in enumerate(self.h_polynomial_coefficients):
            if coeff >= 0:
                result = add(result, multiply(self.encrypted_powers[index],int(coeff)))
            else:
                result = add(result, multiply(self.encrypted_powers[index], curve_order + int(coeff)))
        self.encrypted_h = result

    def delta_shift(self):
        self.delta_shifted_encrypted_p = multiply(self.encrypted_p,self.delta)
        self.delta_shifted_encrypted_p_prime = multiply(self.encrypted_p_prime,self.delta)
        self.delta_shifted_encrypted_h = multiply(self.encrypted_h,self.delta)
        # print(f'The delta value is :{self.delta}')
        # print(f'delta shifted encrypted p: {self.delta_shifted_encrypted_p}')
        # print(f'delta shifted encrypted p prime : {self.delta_shifted_encrypted_p_prime}')
        # print(f'delta shifted encrypted h: {self.delta_shifted_encrypted_h}')

 

In [76]:
# Let's test the 3.6.0 protocol
t_poly_coeffs = calculate_coeff_from_roots([1,2,3,4,5])
p_poly_coeffs = calculate_coeff_from_roots([1,2,3,4,5,6,7])
myVerifier = Verifier_360(t_poly_coeffs,8,field_prime=field_modulus)
# for debugging
# myVerifier.set_secrets(165,2)

myVerifier.calculate_encrypted_powers()
myVerifier.evaluate_unencrypted_target_polynomial()
myProver = Prover_360(myVerifier.encrypted_powers, myVerifier.alpha_shifted_encrypted_powers, t_poly_coeffs,p_poly_coeffs,field_prime=field_properties["bn128"]["field_modulus"])
myProver.calculate_h()
myProver.evaluate_encrypted_h()
myProver.evaluate_encrypted_p_and_p_prime()
myProver.delta_shift()
if myVerifier.check_encrypted_t_h_p_alpha(myProver.delta_shifted_encrypted_h,myProver.delta_shifted_encrypted_p,myProver.delta_shifted_encrypted_p_prime):
    print(f'Everything checks out')

# for debugging
# myProver.evaluate_encrypted_p_and_p_prime_debug(2)


Secret value S is: 6158647537331204230256925749817795024912885018914198724108225360204109386031
Powers of S: [(1, 2), (12366035453080756215758510719779994333885794332964311072532349655908474000599, 2745842577472812745627009419209899960090581875053937899085307338430398968639), (12254367457112042852253154531706967798028220880373038794334348779032682200956, 10100420084418404096298584454484014895143558966472834272052708261160739681093), (12144210159411109891545157257882167049367912106173815182638541505782623277112, 15845974774152092683991823514890392250040267602978588372292195119080826337355), (3690957288009182665879678524121735386574753009554584381846067869435854600965, 10064599841140784118282627744627294145584628507859759356361141782368152053756), (5778446758933567898955777442051248932224258671368535704320653060569817610450, 20910488805550185101248720750138759190119836698481265966911618045886866359050), (5156762637157268121578156440212774552622018603391707505353718703194630915817, 119042

Now we look at implementing the non-interactive version of the protocol and for this, we need to be able to provide alpha and t(s) in encrypted values to the prover. Now that's easy, we can simply use our encryption function, but then what can the prover do with them? So far we have been able to provide a way to multiply two encrypted values. This can be done by using something called the pairing of 2 elliptic curves, which provides a result in a 3rd curve.



In [77]:
# Let's play with the concept of pairing to understand how we can implement the protocol
# We know how to add and multiply in the encrypted thanks to our implementation of 3.6.0 protocol
# But how does that interact with this concept of pairing?
# The first naive approach is to try to use the FQ^2 curve to encrypt alpha and t(s)
# Then see if when we pair them with the other encrypted values (i.e. encrypted p, encrypted h)
# Now if we say FQ-encrypt(2), FQ2-encrypt(3), pair them, see if they are equal to FQ12-encrypt(6)

curve = 'bn128'

if curve == 'optimized_bn128':    
    G1 = optimized_bn128.G1
    G2 = optimized_bn128.G2
    G12 = optimized_bn128.G12
    pairing = optimized_bn128.pairing
    neg = optimized_bn128.neg
    multiply = optimized_bn128.multiply
    eq = optimized_bn128.eq
elif curve == 'bn128':
    G1 = bn128.G1
    G2 = bn128.G2
    G12 = bn128.G12
    pairing = bn128.pairing
    neg = bn128.neg
    multiply = bn128.multiply
    eq = bn128.eq
elif curve == 'bls12_381':
    G1 = bls12_381.G1
    G2 = bls12_381.G2
    G1 = bls12_381.G12
    pairing = bls12_381.pairing
    neg = bls12_381.neg
    multiply = bls12_381.multiply
    eq = bls12_381.eq



# p1 = pairing(G2, G1)
# p2 = pairing(G2, multiply(G1, 2))
# #pairing(neg(G2), G1)

# assert p1 * p1 == p2


# two =  encrypt_over_FQ(2)
# three = encrypt_over_FQ2(3)
# six_paired = pairing(three, two)
# # six = multiply(G12,6)
# # assert eq(six_paired,six)

# # The above does not work, probably because there is a bug in the FQ^12 implementation, but fear not
# # we can still make this work by encrypting the result in FQ and pairing it with encrypted 1...
# six = pairing(G2,encrypt_over_FQ(6))
# assert eq(six_paired,six)

# # let's see if this works with negative values too
# ntwo =  encrypt_over_FQ(-2)
# three = encrypt_over_FQ2(3)
# nsix_paired = pairing(three, ntwo)
# nsix = pairing(G2,encrypt_over_FQ(-6))
# assert eq(nsix_paired,nsix)

# The above tests show that it works, but the pairing operation is really long, so I'm going to comment 
# them out


Back to trying to implement a non-interactive protocol. The first step is to re-use protocol 3.6.0, encrypt t(s) and alpha in FQ2 and finally perform the verification using pairing. A small step towards non-interactivity, but a necessary one.

In [ ]:
# Verifier 3.6.1 

class Verifier_361(Verifier_360):
    def __init__(self, target_polynomial_coefficients, max_order_p_poly, field_prime = field_prime, generator = generator):
        super().__init__(target_polynomial_coefficients, max_order_p_poly, field_prime, generator)
        
    def encrypt_verification_keys(self):
        # This is what t(s) and alpha are called
        # In the next iteration of the protocol, this will be done by a trusted party setup once and for all
        self.encrypted_t = encrypt_over_FQ2(self.evaluated_poly)
        self.encrypted_alpha = encrypt_over_FQ2(self.alpha)
        
    def check_encrypted_t_h_p_alpha(self, encrypted_h:bn128_FQ, encrypted_p:bn128_FQ, encrypted_p_prime:bn128_FQ) -> bool:
        # (f)
        # print(f'Encrypted p is: {encrypted_p}, encrypted p\' is : {encrypted_p_prime} and alpha is :{self.alpha}')
        # print(f'Encrypted p * alpha is :{bn128.multiply(encrypted_p,self.alpha)}')
        # print(f'Encrypted H is: {encrypted_h}')
        encrypted_h_times_encrypted_t = pairing(self.encrypted_t,encrypted_h)
        print(f'Pairing of Encrypted h and encrypted t(s) is : {encrypted_h_times_encrypted_t}') 
        return (eq(encrypted_h_times_encrypted_t,pairing(G2,encrypted_p)) and \
            eq(pairing(G2,encrypted_p_prime), pairing(self.encrypted_alpha, encrypted_p)))

In [80]:
# Let's test the 3.6.1 protocol
t_poly_coeffs = calculate_coeff_from_roots([1,2,3,4,5])
p_poly_coeffs = calculate_coeff_from_roots([1,2,3,4,5,6,7])
myVerifier = Verifier_361(t_poly_coeffs,8,field_prime=field_properties["bn128"]["field_modulus"])
# for debugging
# myVerifier.set_secrets(165,2)

myVerifier.calculate_encrypted_powers()
myVerifier.evaluate_unencrypted_target_polynomial()
myVerifier.encrypt_verification_keys()
myProver = Prover_360(myVerifier.encrypted_powers, myVerifier.alpha_shifted_encrypted_powers, t_poly_coeffs,p_poly_coeffs,field_prime=field_properties["bn128"]["field_modulus"])
myProver.calculate_h()
myProver.evaluate_encrypted_h()
myProver.evaluate_encrypted_p_and_p_prime()
myProver.delta_shift()
if myVerifier.check_encrypted_t_h_p_alpha(myProver.delta_shifted_encrypted_h,myProver.delta_shifted_encrypted_p,myProver.delta_shifted_encrypted_p_prime):
    print(f'Everything checks out')

# for debugging
# myProver.evaluate_encrypted_p_and_p_prime_debug(2)


Secret value S is: 6246461363753864642487640571381854407433804164725800692506840045911087770879
Powers of S: [(1, 2), (13483579295808255299670198997920878083164097833206260784254739809227767936783, 20901489569062708267503264706599669384478393673804798054367964440231047838939), (6140059482788080150747657309042229377427511682877183417761925126916305548315, 9711870991472374132349582246889537254773113709869214060798557976441584210196), (8951109373378063012298344229503794131534026614507503042352007023851406796730, 2918575965123068286956449652703501401663457566395710410935530577870265019971), (21861269215817505688501428000121709965284950923069522405681752195030028918108, 1314855140061054312862213370554125909208144347222661664178408188253144088176), (4879980658931539631188797676420340734603261287996454371482976417049045871837, 8223001331051991425901180637807636874549292256559856449863820872973203006478), (15376248538585787768086804749182089320625443853592800162111411813855663280855, 277791239

AttributeError: 'Verifier_361' object has no attribute 'encrypted_alpha'